In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from keras.utils.vis_utils import plot_model
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CONCATENO

In [ ]:
# caricare a mano solo file options, no underlying
df = []
for file in os.listdir('/'):
  if file.endswith('csv'):
    print('Loading file {0}...'.format(file))
    df.append(pd.read_csv(os.path.join('/', file), sep = ';'))

In [ ]:
df = pd.concat(df, axis = 0)
# df.to_csv('options.csv', index = False)

In [ ]:
df = df.drop(columns=['Expiry_Date','Exercise_Style','Implied_Volatility']) # non tolgo Sigma60

In [ ]:
df = df.dropna(axis=0)
df.shape

(218863, 9)

In [ ]:
call_df = df[df.Flag == 'Call'].drop(['Flag'], axis=1)
put_df = df[df.Flag == 'Put'].drop(['Flag'], axis=1)
#print(call_df)

In [ ]:
# solo ora caricare file underlying
underlying = pd.read_csv('/underlying.csv',sep=';')
print(underlying)

In [ ]:
N_TIMESTEPS = 60

In [ ]:
padded = np.insert(underlying.Close.values, 0, np.array([np.nan] * N_TIMESTEPS))
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]
rolled = np.column_stack((underlying.Date.values[N_TIMESTEPS - 1:], rolled))
price_history = pd.DataFrame(data=rolled)
joined = df.join(price_history.set_index(0), on='Date')
print(joined)

In [ ]:
call_df = joined[joined.Flag == 'Call'].drop(['Flag'], axis=1)
put_df = joined[joined.Flag == 'Put'].drop(['Flag'], axis=1)
call_df = call_df.drop(columns=['Date'])
put_df = put_df.drop(columns=['Date'])
#print(call_df)

In [ ]:
call_X_train_BS, call_X_test_BS, call_y_train_BS, call_y_test_BS = train_test_split(call_df.drop(['Bid', 'Ask'], axis=1),
                                                                        (call_df.Bid + call_df.Ask) / 2,
                                                                        test_size=0.01, random_state=42)
put_X_train_BS, put_X_test_BS, put_y_train_BS, put_y_test_BS = train_test_split(put_df.drop(['Bid', 'Ask'], axis=1),
                                                                    (put_df.Bid+ put_df.Ask) / 2,
                                                                    test_size=0.01, random_state=42)

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['Bid', 'Ask','Sigma60'], axis=1), #qua drop Sigma60-> anche non necessario perché non altera i risultati
                                                                        call_df[['Bid','Ask']],
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['Bid', 'Ask','Sigma60'], axis=1),
                                                                    put_df[['Bid','Ask']],
                                                                    test_size=0.01, random_state=42)
#print(call_X_train)

In [ ]:
call_X_train = call_X_train.values
call_X_test = call_X_test.values
put_X_train = put_X_train.values
put_X_test = put_X_test.values

In [ ]:
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]
put_X_train = [put_X_train[:, -N_TIMESTEPS:].reshape(put_X_train.shape[0], N_TIMESTEPS, 1), put_X_train[:, :4]]
put_X_test = [put_X_test[:, -N_TIMESTEPS:].reshape(put_X_test.shape[0], N_TIMESTEPS, 1), put_X_test[:, :4]]

In [ ]:
layers = 4 
features = 4 
n_batch = 128 
n_epochs = 40

METRICS QUI

In [ ]:
call_model = load_model('/20221003-call-lstm-v2.h5')
put_model = load_model('/20221003-put-lstm-v2.h5')

In [ ]:
from scipy.stats import norm
def black_scholes(row):
    S = row.Underlying_Price
    X = row.Strike_Price
    T = row.Maturity / 365
    r = row.Rf_Rate / 100
    σ = row.Sigma60
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return C
def black_scholes_put(row):
    S = row.Underlying_Price
    X = row.Strike_Price
    T = row.Maturity / 365
    r = row.Rf_Rate / 100
    σ = row.Sigma60
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    P  = norm.cdf(-d2) * X * np.exp(-r * T) - S * norm.cdf(-d1)
    return P

In [ ]:
def error_metrics(actual, predicted):
    diff = actual - predicted
    mse = np.mean(np.square(diff))
    rel = diff / actual
    mpe = 100 * np.median(rel)
    aape = 100 * np.mean(np.abs(rel))
    mape = 100 * np.median(np.abs(rel))
    pe5 = 100 * sum(np.abs(rel) < 0.05) / rel.shape[0]
    pe10 = 100 * sum(np.abs(rel) < 0.10) / rel.shape[0]
    pe20 = 100 * sum(np.abs(rel) < 0.20) / rel.shape[0]
    return [mse, bias, aape, mape, pe5, pe10, pe20]

In [ ]:
call_y_pred = call_model.predict(call_X_test)
put_y_pred = put_model.predict(put_X_test)
line1 = error_metrics(np.mean(call_y_test, axis=1), np.mean(call_y_pred, axis=1))
line2 = error_metrics(np.mean(put_y_test, axis=1), np.mean(put_y_pred, axis=1))
call_train_pred = call_model.predict(call_X_train)
put_train_pred = put_model.predict(put_X_train)
line1.insert(0, np.mean(np.square(np.diff(call_y_train, axis=1) - np.diff(call_train_pred, axis=1))))
line2.insert(0, np.mean(np.square(np.diff(put_y_train, axis=1) - np.diff(put_train_pred, axis=1))))
line3 = error_metrics(call_y_test_BS, black_scholes(call_X_test_BS))
line4 = error_metrics(put_y_test_BS, black_scholes_put(put_X_test_BS))
line3.insert(0, np.mean(np.square(call_y_train_BS - black_scholes(call_X_train_BS))))
line4.insert(0, np.mean(np.square(put_y_train_BS - black_scholes_put(put_X_train_BS))))
for line in (line1, line2, line3, line4):
    print('& {:.2f} & {:.2f} & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% & {:.2f}\% \\\\'.format(*line))

& 720.19 & 85.79 & 1.88\% & 35.32\% & 5.71\% & 46.85\% & 58.70\% & 65.37\% \\
& 167.69 & 61.02 & -0.40\% & 22.89\% & 2.69\% & 63.30\% & 73.40\% & 77.28\% \\
& 6037.64 & 6549.02 & 93.98\% & 63.59\% & 93.98\% & 15.93\% & 21.67\% & 27.96\% \\
& 3820.32 & 3824.51 & 56.02\% & 52.51\% & 56.02\% & 32.82\% & 38.50\% & 41.93\% \\
